In [ ]:
import multiprocessing
import tqdm

import numpy as np
import scipy.stats as st
import scipy
import numba
import math

#import biocircuits

# Plotting modules
import bokeh.io
import bokeh.plotting
from bokeh.models import LinearColorMapper, ColorBar
#from bokeh.io import export_svgs

bokeh.io.output_notebook()

# Line profiler (can install with conda install line_profiler)
# %load_ext line_profiler

## how two nodes that process the same inputs affect the decision boundary - varying ct

In [ ]:
def Sequestration_rhs(x,t,a1,b1,a2,b2,g1,g2,d,ct):
    S1, A1, S2, A2, C1, C2 = x
    C = ct - C1 - C2
    return np.array(
        [
            a1 - d*S1 - g1*A1*S1 - g2*S1*C,
            b1 - d*A1 - g1*A1*S1,
            a2 - d*S2 - g1*A2*S2 - g2*S2*C,
            b2 - d*A2 - g1*A2*S2,
            g2*S1*C - d*C1,
            g2*S2*C - d*C2,
        ]
    )

g1 = 1000.
g2 = 10.
d = 1.
ct = [0.2, 0.6, 1.]
a2 = 0
b2 = 0.5


tN = 100
N = 21
x1 = np.linspace(0,1,N)
x2 = np.linspace(0,1,N)

x0 = np.array([0., 0., 0., 0., 0., 0.])
t = np.linspace(0,5,tN)

output1 = np.zeros((N,N,3))
output2 = np.zeros((N,N,3))

for k, ctk in enumerate(ct):
    for i, x1i in enumerate(x1):
        for j, x2j in enumerate(x2):
            x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(x1i, x2j, a2 + x1i + x2j, b2,  g1, g2, d, ctk))
            output1[j,i,k] = x.transpose()[4,-1]/ctk # 0 (s1) 4 (c1)
            output2[j,i,k] = x.transpose()[5,-1]/ctk # 0 (s1) 4 (c1)

# Set up plots
fig_size = (150,115)
#fig_size = (130,115)
x_range = (x1[0],x1[-1])
y_range = (x2[0],x2[-1])

palette = bokeh.palettes.grey(15)[::-1]
palette = bokeh.palettes.Blues9[::-1]

plots = []
for i in range(6):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k, itr in enumerate(ct):
    z0 = np.c_[output1[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output2[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+3].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

color = LinearColorMapper(palette = palette, low = z0.min(), high = z0.max())
cb = ColorBar(color_mapper = color, location = (0,0), width=5)
#plots[0].add_layout(cb, 'right')

bokeh.io.show(bokeh.layouts.row(plots[0:3])) # Top C1
bokeh.io.show(bokeh.layouts.row(plots[3:6])) # botton C2

## Linear decision without competition -- varying $\gamma_1$

In [ ]:
g1 = [10, 100, 1000.]
g2 = 10.
d = 1.
ct = 1/5
a2 = 0
b2 = 0.5


tN = 100
N = 21
x1 = np.linspace(0,1,N)
x2 = np.linspace(0,1,N)

x0 = np.array([0., 0., 0., 0., 0., 0.])
t = np.linspace(0,5,tN)

output1 = np.zeros((N,N,3))
output2 = np.zeros((N,N,3))

for k, g1k in enumerate(g1):
    for i, x1i in enumerate(x1):
        for j, x2j in enumerate(x2):
            x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(x1i, x2j, 0, 0,  g1k, g2, d, ct))
            output1[j,i,k] = x.transpose()[4,-1]/ct # 0 (s1) 4 (c1)
            x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(0, 0, a2 + x1i + x2j, b2,  g1k, g2, d, ct))
            output2[j,i,k] = x.transpose()[5,-1]/ct # 0 (s1) 4 (c1)

# Set up plots
fig_size = (150,115)
#fig_size = (130,115)
x_range = (x1[0],x1[-1])
y_range = (x2[0],x2[-1])

palette = bokeh.palettes.grey(15)[::-1]
palette = bokeh.palettes.Blues9[::-1]

plots = []
for i in range(6):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k, itr in enumerate(g1):
    z0 = np.c_[output1[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output2[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+3].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

bokeh.io.show(bokeh.layouts.row(plots[0:3])) # Top C1
bokeh.io.show(bokeh.layouts.row(plots[3:6])) # botton C2

## how two nodes that process the same inputs affect the decision boundary - varying gamma1

In [ ]:
g1 = [10, 100, 1000.]
g2 = 10.
d = 1.
ct = 1/5
a2 = 0
b2 = 0.5


tN = 100
N = 21
x1 = np.linspace(0,1,N)
x2 = np.linspace(0,1,N)

x0 = np.array([0., 0., 0., 0., 0., 0.])
t = np.linspace(0,5,tN)

output1 = np.zeros((N,N,3))
output2 = np.zeros((N,N,3))

for k, g1k in enumerate(g1):
    for i, x1i in enumerate(x1):
        for j, x2j in enumerate(x2):
            x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(x1i, x2j, a2 + x1i + x2j, b2,  g1k, g2, d, ct))
            output1[j,i,k] = x.transpose()[4,-1]/ct # 0 (s1) 4 (c1)
            output2[j,i,k] = x.transpose()[5,-1]/ct # 0 (s1) 4 (c1)

# Set up plots
fig_size = (150,115)
#fig_size = (130,115)
x_range = (x1[0],x1[-1])
y_range = (x2[0],x2[-1])

palette = bokeh.palettes.grey(15)[::-1]
palette = bokeh.palettes.Blues9[::-1]

plots = []
for i in range(6):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k, itr in enumerate(g1):
    z0 = np.c_[output1[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output2[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+3].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

bokeh.io.show(bokeh.layouts.row(plots[0:3])) # Top C1
bokeh.io.show(bokeh.layouts.row(plots[3:6])) # botton C2

## Modeling three node classifier - varying ct

In [ ]:
def Sequestration_rhs2(x,t,a1,b1,a2,b2,a3,b3,g1,g2,d,ct, w1, w2):
    S1, A1, S2, A2, S3, A3, C1, C2, C3 = x
    C = ct - C1 - C2 - C3
    return np.array(
        [
            a1 - d*S1 - g1*A1*S1 - g2*S1*C,
            b1 - d*A1 - g1*A1*S1,
            a2 - d*S2 - g1*A2*S2 - g2*S2*C,
            b2 - d*A2 - g1*A2*S2,
            a3 - d*S3 - g1*A3*S3 - g2*S3*C + w1*C1 + w2*C2,
            b3 - d*A3 - g1*A3*S3,
            g2*S1*C - d*C1,
            g2*S2*C - d*C2,
            g2*S3*C - d*C3,
        ]
    )

x0 = np.array([0., 0., 0., 0., 0., 0., 0., 0., 0.])
a1 = 1
b1 = 0.5
a2 = 0
b2 = 0
a3 = 0
b3 = 0

g1 = 100
g2 = 100

d = 1
ct = 1

w3 = (1.,1.)

t = np.linspace(0,15,51)
x = scipy.integrate.odeint(Sequestration_rhs2, x0, t, args=(a1, b1, a2, b2, a3, b3, g1, g2, d, ct, w3[0], w3[1]))
x = x.transpose()

# Set up plots
fig_size = [150, 115] # for paper size (144, 115) # for larger size (144,144)
x_range = (0, 6)
y_range = (0, 0.61)

plots = []
for i in range(1):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          #x_range=x_range,y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"     #background_fill_color="#fafafa"


plots[0].line(t,x[0,:], line_width=2, color="black")
plots[0].line(t,x[1,:], line_width=2, color="orange")

plots[0].circle(t[10:41:10],x[0,10:41:10], line_width=2, color="black")


bokeh.io.show(bokeh.layouts.row(plots))

In [ ]:
g1 = 1000.
g2 = 10.
d = 1.
ct = [0.2, 0.6, 1.]
a1 = 0
b1 = 1.2 # 1.2
a2 = 0.7 # 0.7
b2 = 0
a3 = 0
b3 = 0.15
w3 = (4.,4.) # 6

tN = 100
N = 21 #10 or 21
x1 = np.linspace(0,1,N)
x2 = np.linspace(0,1,N)

x0 = np.array([0., 0., 0., 0., 0., 0., 0., 0., 0.])
t = np.linspace(0,5,tN)

output1 = np.zeros((N,N,3))
output2 = np.zeros((N,N,3))
output3 = np.zeros((N,N,3))

for k, ctk in enumerate(ct):
    for i, x1i in enumerate(x1):
        for j, x2j in enumerate(x2):
            x = scipy.integrate.odeint(Sequestration_rhs2, x0, t, args=(x1i + x2j, b1, a2, x1i + x2j, a3, b3,  g1, g2, d, ctk, w3[0], w3[1]))
            output1[j,i,k] = x.transpose()[6,-1]/ctk # C1
            output2[j,i,k] = x.transpose()[7,-1]/ctk # C2
            output3[j,i,k] = x.transpose()[8,-1]/ctk # C3

# Set up plots
fig_size = (150,115)
#fig_size = (130,115)
x_range = (x1[0],x1[-1])
y_range = (x2[0],x2[-1])

palette = bokeh.palettes.grey(15)[::-1]
palette = bokeh.palettes.Blues9[::-1]

plots = []
for i in range(9):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k, itr in enumerate(ct):
    z0 = np.c_[output1[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output2[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+3].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output3[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+6].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

bokeh.io.show(bokeh.layouts.row(plots[0:3])) # Top C1
bokeh.io.show(bokeh.layouts.row(plots[3:6])) # Center C2
bokeh.io.show(bokeh.layouts.row(plots[6:9])) # Botom C3

## Three node band classifier - varying ct

In [ ]:
def Sequestration_rhs20(x,t,a1,b1,a2,b2,a3,b3,g1,g2,d,ct, w1, w2):
    S1, A1, S2, A2, S3, A3, C1, C2, C3 = x
    C = ct - C1 - C2 - C3
    return np.array(
        [
            a1 - d*S1 - g1*A1*S1 - g2*S1*C,
            b1 - d*A1 - g1*A1*S1,
            a2 - d*S2 - g1*A2*S2 - g2*S2*C,
            b2 - d*A2 - g1*A2*S2,
            a3 - d*S3 - g1*A3*S3 - g2*S3*C ,
            b3 - d*A3 - g1*A3*S3 + w1*C1 + w2*C2,
            g2*S1*C - d*C1,
            g2*S2*C - d*C2,
            g2*S3*C - d*C3,
        ]
    )

g1 = 1000.
g2 = 10.
d = 1.
ct = [0.2, 0.6, 1.]
a1 = 0
b1 = 1.2 # 1.2
a2 = 0.7 # 0.7
b2 = 0
a3 = 0.2
b3 = 0.
w3 = (1.,1.) # 6

tN = 100
N = 21 #10 or 21
x1 = np.linspace(0,1,N)
x2 = np.linspace(0,1,N)

x0 = np.array([0., 0., 0., 0., 0., 0., 0., 0., 0.])
t = np.linspace(0,5,tN)

output1 = np.zeros((N,N,3))
output2 = np.zeros((N,N,3))
output3 = np.zeros((N,N,3))

for k, ctk in enumerate(ct):
    for i, x1i in enumerate(x1):
        for j, x2j in enumerate(x2):
            x = scipy.integrate.odeint(Sequestration_rhs20, x0, t, args=(x1i + x2j, b1, a2, x1i + x2j, a3, b3,  g1, g2, d, ctk, w3[0], w3[1]))
            output1[j,i,k] = x.transpose()[6,-1]/ctk # C1
            output2[j,i,k] = x.transpose()[7,-1]/ctk # C2
            output3[j,i,k] = x.transpose()[8,-1]/ctk # C3

# Set up plots
fig_size = (150,115)
#fig_size = (130,115)
x_range = (x1[0],x1[-1])
y_range = (x2[0],x2[-1])

palette = bokeh.palettes.grey(15)[::-1]
palette = bokeh.palettes.Blues9[::-1]

plots = []
for i in range(9):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k, itr in enumerate(ct):
    z0 = np.c_[output1[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output2[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+3].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output3[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+6].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

bokeh.io.show(bokeh.layouts.row(plots[0:3])) # Top C1
bokeh.io.show(bokeh.layouts.row(plots[3:6])) # Center C2
bokeh.io.show(bokeh.layouts.row(plots[6:9])) # Botom C3

## Modeling four node classifier - varying ct

In [ ]:
def Sequestration_rhs3(x,t,a1,b1,a2,b2,a3,b3,a4,b4,g1,g2,d,ct, w4):
    S1, A1, S2, A2, S3, A3, S4, A4, C1, C2, C3, C4 = x
    C = ct - C1 - C2 - C3 - C4
    return np.array(
        [
            a1 - d*S1 - g1*A1*S1 - g2*S1*C,
            b1 - d*A1 - g1*A1*S1,
            a2 - d*S2 - g1*A2*S2 - g2*S2*C,
            b2 - d*A2 - g1*A2*S2,
            a3 - d*S3 - g1*A3*S3 - g2*S3*C,
            b3 - d*A3 - g1*A3*S3,
            a4 - d*S4 - g1*A4*S4 - g2*S4*C,
            b4 - d*A4 - g1*A4*S4 + w4[1]*C1 + w4[2]*C2 + w4[3]*C3,
            g2*S1*C - d*C1,
            g2*S2*C - d*C2,
            g2*S3*C - d*C3,
            g2*S4*C - d*C4,
        ]
    )

x0 = np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
a1 = 1
b1 = 0.5
a2 = 0
b2 = 0
a3 = 0
b3 = 0
a4 = 0
b4 = 0

g1 = 100
g2 = 100

d = 1
ct = 1

w4 = [0.,1.,1.,1.]

t = np.linspace(0,15,51)
x = scipy.integrate.odeint(Sequestration_rhs3, x0, t, args=(a1, b1, a2, b2, a3, b3, a4, b4, g1, g2, d, ct, w4))
x = x.transpose()

# Set up plots
fig_size = [150, 115] # for paper size (144, 115) # for larger size (144,144)
x_range = (0, 6)
y_range = (0, 0.61)

plots = []
for i in range(1):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          #x_range=x_range,y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"     #background_fill_color="#fafafa"


plots[0].line(t,x[0,:], line_width=2, color="black")
plots[0].line(t,x[1,:], line_width=2, color="orange")

plots[0].circle(t[10:41:10],x[0,10:41:10], line_width=2, color="black")


bokeh.io.show(bokeh.layouts.row(plots))

In [ ]:
def Sequestration_rhs3(x,t,a1,b1,a2,b2,a3,b3,a4,b4,g1,g2,d,ct, w4):
    S1, A1, S2, A2, S3, A3, S4, A4, C1, C2, C3, C4 = x
    C = ct - C1 - C2 - C3 - C4
    return np.array(
        [
            a1 - d*S1 - g1*A1*S1 - g2*S1*C,
            b1 - d*A1 - g1*A1*S1,
            a2 - d*S2 - g1*A2*S2 - g2*S2*C,
            b2 - d*A2 - g1*A2*S2,
            a3 - d*S3 - g1*A3*S3 - g2*S3*C,
            b3 - d*A3 - g1*A3*S3,
            a4 - d*S4 - g1*A4*S4 - g2*S4*C,
            b4 - d*A4 - g1*A4*S4 + w4[1]*C1 + w4[2]*C2 + w4[3]*C3,
            g2*S1*C - d*C1,
            g2*S2*C - d*C2,
            g2*S3*C - d*C3,
            g2*S4*C - d*C4,
        ]
    )

g1 = 1000.
g2 = 10.
d = 1.
ct = [0.2, 0.6, 1.]
#ct = 0.6
#a1 = [0.2, 0.3, 0.4]#0.5
a1 = 0.4
b1 = 0. # 1.2
a2 = 0 # 0.7
b2 = 0.8
a3 = 0.
b3 = 0.8
a4 = 0.3
b4 = 0.0
w3 = (0.,1,1.,1) # 6

tN = 100
N = 21 #10 or 21
x1 = np.linspace(0,1,N)
x2 = np.linspace(0,1,N)

x0 = np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
t = np.linspace(0,5,tN)

output1 = np.zeros((N,N,3))
output2 = np.zeros((N,N,3))
output3 = np.zeros((N,N,3))
output4 = np.zeros((N,N,3))

for k, ctk in enumerate(ct):
    for i, x1i in enumerate(x1):
        for j, x2j in enumerate(x2):
            x = scipy.integrate.odeint(Sequestration_rhs3, x0, t, args=(a1, 0.5*x1i + 0.5*x2j+ b1, a2 + 1.5*x1i, 0.5*x2j +b2,  1.5*x2j + a3, 0.5*x1i +b3, a4, b4, g1, g2, d, ctk, w3))
            output1[j,i,k] = x.transpose()[8,-1]/ctk # C1
            output2[j,i,k] = x.transpose()[9,-1]/ctk # C2
            output3[j,i,k] = x.transpose()[10,-1]/ctk # C3
            output4[j,i,k] = x.transpose()[11,-1]/ctk # C4

# Set up plots
fig_size = (150,115)
#fig_size = (130,115)
x_range = (x1[0],x1[-1])
y_range = (x2[0],x2[-1])

palette = bokeh.palettes.grey(15)[::-1]
palette = bokeh.palettes.Blues9[::-1]

plots = []
for i in range(12):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k, itr in enumerate(ct):
    z0 = np.c_[output1[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output2[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+3].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output3[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+6].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output4[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1
    plots[k+9].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

bokeh.io.show(bokeh.layouts.row(plots[0:3])) # Top C1
bokeh.io.show(bokeh.layouts.row(plots[3:6])) # Center C2
bokeh.io.show(bokeh.layouts.row(plots[6:9])) # Botom C3
bokeh.io.show(bokeh.layouts.row(plots[9:12])) # Botom C4

In [ ]:
output4[:,:,0].max()

0.3782083941659414

## Linear classifier and AND gate

In [ ]:
def Sequestration_rhs4(x,t,a1,b1,a2,b2,a3,b3,g1,g2,g,d,ct, w4):
    S1, A1, S2, A2, S3, A3, P1, P2, P3, C1, C2, C3, CP = x
    C = ct - C1 - C2 - C3
    return np.array(
        [
            a1 - d*S1 - g1*A1*S1 - g2*S1*C,
            b1 - d*A1 - g1*A1*S1,
            a2 - d*S2 - g1*A2*S2 - g2*S2*C,
            b2 - d*A2 - g1*A2*S2,
            a3 - d*S3 - g1*A3*S3 - g2*S3*C,
            b3 - d*A3 - g1*A3*S3,
            w4[1]*C1 - d*P1 - g*P1*P2*P3,
            w4[2]*C2 - d*P2 - g*P1*P2*P3,
            w4[3]*C3 - d*P3 - g*P1*P2*P3,
            g2*S1*C - d*C1,
            g2*S2*C - d*C2,
            g2*S3*C - d*C3,
            g*P1*P2*P3 - d*CP,
        ]
    )

In [ ]:
g1 = 100.
g2 = 10.
g = 10.0
d = 1.
ct = [0.2, 0.6, 1.]
a1 = 0.
b1 = 0.4 # 1.2
a2 = 1. # 0.7
b2 = 0.
a3 = 1.
b3 = 0.

w3 = (0.,10,10.,10) # 6

tN = 100
N = 21 #10 or 21
x1 = np.linspace(0,1,N)
x2 = np.linspace(0,1,N)

x0 = np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
t = np.linspace(0,5,tN)

output1 = np.zeros((N,N,3))
output2 = np.zeros((N,N,3))
output3 = np.zeros((N,N,3))
output4 = np.zeros((N,N,3))

for k, ctk in enumerate(ct):
    for i, x1i in enumerate(x1):
        for j, x2j in enumerate(x2):
            x = scipy.integrate.odeint(Sequestration_rhs4, x0, t, args=(a1 + x1i + x2j, b1, a2 + x1i, 2.5*x2j +b2,  x2j + a3, 2.5*x1i + b3, g1, g2, g, d, ctk, w4))
            output1[j,i,k] = x.transpose()[9,-1]/ctk # C1
            output2[j,i,k] = x.transpose()[10,-1]/ctk # C2
            output3[j,i,k] = x.transpose()[11,-1]/ctk # C3
            output4[j,i,k] = x.transpose()[12,-1] # C4

# Set up plots
fig_size = (150,115)
#fig_size = (130,115)
x_range = (x1[0],x1[-1])
y_range = (x2[0],x2[-1])

palette = bokeh.palettes.grey(15)[::-1]
palette = bokeh.palettes.Blues9[::-1]

plots = []
for i in range(12):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k, itr in enumerate(ct):
    z0 = np.c_[output1[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output2[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+3].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output3[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+6].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output4[:,:,k], np.zeros(N)]
    #z0[-1,-1] = .2
    plots[k+9].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

bokeh.io.show(bokeh.layouts.row(plots[0:3])) # Top C1
bokeh.io.show(bokeh.layouts.row(plots[3:6])) # Center C2
bokeh.io.show(bokeh.layouts.row(plots[6:9])) # Botom C3
bokeh.io.show(bokeh.layouts.row(plots[9:12])) # Botom C4

In [ ]:
output4[:,:,k].max()

0.14595892436757169

In [ ]:
def Sequestration_rhs(x,t,a1,b1,a2,b2,g1,g2,d,ct):
    S1, A1, S2, A2, C1, C2 = x
    C = ct - C1 - C2
    return np.array(
        [
            a1 - d*S1 - g1*A1*S1 - g2*S1*C,
            b1 - d*A1 - g1*A1*S1,
            a2 - d*S2 - g1*A2*S2 - g2*S2*C,
            b2 - d*A2 - g1*A2*S2,
            g2*S1*C - d*C1,
            g2*S2*C - d*C2,
        ]
    )

x0 = np.array([0., 0., 0., 0., 0., 0.])
a1 = 1
b1 = 0.5
a2 = 0
b2 = 0

g1 = 100
g2 = 100

d = 1
ct = 1

t = np.linspace(0,15,51)
x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(a1, b1, a2, b2,  g1, g2, d, ct))
x = x.transpose()

# Set up plots
fig_size = [150, 115] # for paper size (144, 115) # for larger size (144,144)
x_range = (0, 6)
y_range = (0, 0.61)

plots = []
for i in range(1):
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          #x_range=x_range,y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"     #background_fill_color="#fafafa"


plots[0].line(t,x[0,:], line_width=2, color="black")
plots[0].line(t,x[1,:], line_width=2, color="orange")

plots[0].circle(t[10:41:10],x[0,10:41:10], line_width=2, color="black")


bokeh.io.show(bokeh.layouts.row(plots))

## How does the strength of sequestration affect the decision boundary

In [ ]:
g1 = [10,100,1000] # sequestration rate
g2 = 10 # active complex rate
d = 1
ct = 1/5
a2 = 0
b2 = 0


tN = 100
N = 21
x1 = np.linspace(0,1,N)
x2 = np.linspace(0,1,N)

x0 = np.array([0., 0., 0., 0., 0., 0.])
t = np.linspace(0,5,tN)

output1 = np.zeros((N,N,3))
output2 = np.zeros((N,N,3))

for k, g1k in enumerate(g1):
    for i, x1i in enumerate(x1):
        for j, x2j in enumerate(x2):
            x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(x1i, x2j, a2, b2,  g1k, g2, d, ct))
            output1[j,i,k] = x.transpose()[0,-1] # 0 (s1) 4 (c1)
            output2[j,i,k] = x.transpose()[4,-1]/ct # 0 (s1) 4 (c1)

# Set up plots
fig_size = (150,115)
#fig_size = (130,115)
x_range = (x1[0],x1[-1])
y_range = (x2[0],x2[-1])

palette = bokeh.palettes.grey(15)[::-1]
palette = bokeh.palettes.Blues9[::-1]

plots = []
for i in range(6):
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k, itr in enumerate(g1):
    z0 = np.c_[output1[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output2[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+3].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

bokeh.io.show(bokeh.layouts.row(plots[0:3])) # Top S1
bokeh.io.show(bokeh.layouts.row(plots[3:6])) # botton C1

## Tuning the effect of sigma factor 2 in the decision boundary

In [ ]:
g1 = 100.
g2 = 10
d = 1
ct = 1/5
a2 = [0, 0.5, 1]
b2 = 0


N = 21
x1 = np.linspace(0,1,N)
x2 = np.linspace(0,1,N)

x0 = np.array([0., 0., 0., 0., 0., 0.])
t = np.linspace(0,5,tN)

output1 = np.zeros((N,N,3))
output2 = np.zeros((N,N,3))

for k, a2k in enumerate(a2):
    for i, x1i in enumerate(x1):
        for j, x2j in enumerate(x2):
            x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(x1i, x2j, a2k, b2,  g1, g2, d, ct))
            output1[j,i,k] = x.transpose()[0,-1] # 0 (s1) 4 (c1)
            output2[j,i,k] = x.transpose()[4,-1]/ct # 0 (s1) 4 (c1)

# Set up plots
fig_size = (150,115)
#fig_size = (130,115)
x_range = (x1[0],x1[-1])
y_range = (x2[0],x2[-1])

palette = bokeh.palettes.grey(15)[::-1]
palette = bokeh.palettes.Blues9[::-1]

plots = []
for i in range(6):
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k, itr in enumerate(a2):
    z0 = np.c_[output1[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output2[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+3].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

bokeh.io.show(bokeh.layouts.row(plots[0:3])) # Top S1
bokeh.io.show(bokeh.layouts.row(plots[3:6])) # botton C1

## The effect of anti-sigma factor 2

In [ ]:
g1 = 100.
g2 = 10
d = 1
ct = 1/5
a2 = 0.5
b2 = [0, 0.5, 1]


N = 21
x1 = np.linspace(0,1,N)
x2 = np.linspace(0,1,N)

x0 = np.array([0., 0., 0., 0., 0., 0.])
t = np.linspace(0,5,tN)

output1 = np.zeros((N,N,3))
output2 = np.zeros((N,N,3))

for k, b2k in enumerate(b2):
    for i, x1i in enumerate(x1):
        for j, x2j in enumerate(x2):
            x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(x1i, x2j, a2, b2k,  g1, g2, d, ct))
            output1[j,i,k] = x.transpose()[0,-1] # 0 (s1) 4 (c1)
            output2[j,i,k] = x.transpose()[4,-1]/ct # 0 (s1) 4 (c1)

# Set up plots
fig_size = (150,115)
#fig_size = (130,115)
x_range = (x1[0],x1[-1])
y_range = (x2[0],x2[-1])

palette = bokeh.palettes.grey(15)[::-1]
palette = bokeh.palettes.Blues9[::-1]

plots = []
for i in range(6):
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k, itr in enumerate(b2):
    z0 = np.c_[output1[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output2[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+3].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

bokeh.io.show(bokeh.layouts.row(plots[0:3])) # Top S1
bokeh.io.show(bokeh.layouts.row(plots[3:6])) # botton C1

## Effect of sequestration rate 2 on the decision boundary

In [ ]:
g1 = 100 #[10,100,1000]
g2 = [1,10,100]
d = 1
ct = 1/5
a2 = 0
b2 = 0



N = 21
x1 = np.linspace(0,1,N)
x2 = np.linspace(0,1,N)

x0 = np.array([0., 0., 0., 0., 0., 0.])
t = np.linspace(0,5,tN)

output1 = np.zeros((N,N,3))
output2 = np.zeros((N,N,3))

for k, g2k in enumerate(g2):
    for i, x1i in enumerate(x1):
        for j, x2j in enumerate(x2):
            x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(x1i, x2j, a2, b2,  g1, g2k, d, ct))
            output1[j,i,k] = x.transpose()[0,-1] # 0 (s1) 4 (c1)
            output2[j,i,k] = x.transpose()[4,-1]/ct # 0 (s1) 4 (c1)

# Set up plots
fig_size = (150,115)
#fig_size = (130,115)
x_range = (x1[0],x1[-1])
y_range = (x2[0],x2[-1])

palette = bokeh.palettes.grey(15)[::-1]
palette = bokeh.palettes.Blues9[::-1]

plots = []
for i in range(6):
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k, itr in enumerate(g2):
    z0 = np.c_[output1[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output2[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+3].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

bokeh.io.show(bokeh.layouts.row(plots[0:3])) # Top S1
bokeh.io.show(bokeh.layouts.row(plots[3:6])) # botton C1

## Weight 1 tuning decision boundary

In [ ]:
g1 = 100
g2 = 10
d = 1
ct = 1/5
a2 = 0
b2 = 0

w1 = [0.5, 1, 2]

N = 21
x1 = np.linspace(0,1,N)
x2 = np.linspace(0,1,N)

x0 = np.array([0., 0., 0., 0., 0., 0.])
t = np.linspace(0,5,tN)

output1 = np.zeros((N,N,3))
output2 = np.zeros((N,N,3))

for k, w1k in enumerate(w1):
    for i, x1i in enumerate(x1):
        for j, x2j in enumerate(x2):
            x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(w1k*x1i, x2j, a2, b2,  g1, g2, d, ct))
            output1[j,i,k] = x.transpose()[0,-1] # 0 (s1) 4 (c1)
            output2[j,i,k] = x.transpose()[4,-1]/ct # 0 (s1) 4 (c1)

# Set up plots
fig_size = (150,115)
#fig_size = (130,115)
x_range = (x1[0],x1[-1])
y_range = (x2[0],x2[-1])

palette = bokeh.palettes.grey(15)[::-1]
palette = bokeh.palettes.Blues9[::-1]

plots = []
for i in range(6):
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k, itr in enumerate(w1):
    z0 = np.c_[output1[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output2[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+3].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

bokeh.io.show(bokeh.layouts.row(plots[0:3])) # Top S1
bokeh.io.show(bokeh.layouts.row(plots[3:6])) # botton C1

## Weight 1 tuning in shared resources

In [ ]:
g1 = 100
g2 = 10
d = 1
ct = 1/5
a2 = 0.5
b2 = 0

w1 = [0.5, 1, 2]

N = 21
x1 = np.linspace(0,1,N)
x2 = np.linspace(0,1,N)

x0 = np.array([0., 0., 0., 0., 0., 0.])
t = np.linspace(0,5,tN)

output1 = np.zeros((N,N,3))
output2 = np.zeros((N,N,3))

for k, w1k in enumerate(w1):
    for i, x1i in enumerate(x1):
        for j, x2j in enumerate(x2):
            x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(w1k*x1i, x2j, a2, b2,  g1, g2, d, ct))
            output1[j,i,k] = x.transpose()[0,-1] # 0 (s1) 4 (c1)
            output2[j,i,k] = x.transpose()[4,-1]/ct # 0 (s1) 4 (c1)

# Set up plots
fig_size = (150,115)
#fig_size = (130,115)
x_range = (x1[0],x1[-1])
y_range = (x2[0],x2[-1])

palette = bokeh.palettes.grey(15)[::-1]
palette = bokeh.palettes.Blues9[::-1]

plots = []
for i in range(6):
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k, itr in enumerate(w1):
    z0 = np.c_[output1[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output2[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+3].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

bokeh.io.show(bokeh.layouts.row(plots[0:3])) # Top S1
bokeh.io.show(bokeh.layouts.row(plots[3:6])) # botton C1

## Effects of cT in the linear decision boundary

In [ ]:
g1 = 100
g2 = 10
d = 1
ct = [0.2, 0.5, 0.8]
a2 = 0
b2 = 0

#w1 = [0.5, 1, 2]

N = 21
x1 = np.linspace(0,1,N)
x2 = np.linspace(0,1,N)

x0 = np.array([0., 0., 0., 0., 0., 0.])
t = np.linspace(0,5,tN)

output1 = np.zeros((N,N,3))
output2 = np.zeros((N,N,3))

for k, ctk in enumerate(ct):
    for i, x1i in enumerate(x1):
        for j, x2j in enumerate(x2):
            x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(x1i, x2j, a2, b2,  g1, g2, d, ctk))
            output1[j,i,k] = x.transpose()[0,-1] # 0 (s1) 4 (c1)
            output2[j,i,k] = x.transpose()[4,-1]/ctk # 0 (s1) 4 (c1)

# Set up plots
fig_size = (150,115)
#fig_size = (130,115)
x_range = (x1[0],x1[-1])
y_range = (x2[0],x2[-1])

palette = bokeh.palettes.grey(15)[::-1]
palette = bokeh.palettes.Blues9[::-1]

plots = []
for i in range(6):
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k, itr in enumerate(ct):
    z0 = np.c_[output1[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output2[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+3].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

bokeh.io.show(bokeh.layouts.row(plots[0:3])) # Top S1
bokeh.io.show(bokeh.layouts.row(plots[3:6])) # botton C1